In [1]:
import pandas as pd
import numpy as np
import cvxpy as cvx
import random
import time
from __future__ import print_function
from builtins import input

from sporco.admm import bpdn
from sporco import util
from sporco import plot
plot.config_notebook_plotting()

from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold

from scipy import stats
from scipy.stats import spearmanr

import warnings
warnings.filterwarnings("ignore")



table = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/starch-sanchez-v3/matrices/A6.pkl")
iHiVred = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/starch-sanchez-v3/matrices/iHiV5.pkl")
iHiV = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/starch-sanchez-v3/matrices/iHiV6.pkl")
VHred = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/starch-sanchez-v3/matrices/VH4.pkl")
tableg = table.T

B = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/starch-sanchez-v3/matrices/B6.pkl")
Bdf = pd.DataFrame(B)

VH = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/starch-sanchez-v3/matrices/VH6.pkl")
tableg = table.T
species_ones = [] 
for i in range(6):
    species = i
    species_ones.append(list(tableg[tableg[species]==1].index))
    
    
species_zeros = []
for i in range(6):
    species = i
    species_zeros.append(list(tableg[tableg[species]==0].index))                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       


In [2]:
iHiVred.shape

(32, 32)

In [3]:
laml = [1e-30,100e-30,1e-20,1e-10,1e-5,.01,.05,.06,.0650,.0675,.07,0.08,0.1,0.13,0.15,0.18,0.2,0.5,1.0,5.0,10.0]
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error as mape
g = np.array([[1, 0], [-1, 1]])

for i in range(4):
    g = np.block([[g, np.zeros(g.shape)], [-g, g]])

Gred = g
Xred = np.linalg.inv(Gred)

import matplotlib.font_manager as fm
 
gs_font = fm.FontProperties(fname='gillsans.ttf', size=15, weight='bold')
light_grey=(0.5,0.5,0.5)


import matplotlib.patches as mpatches

In [4]:
Xred.shape

(32, 32)

In [5]:
sp_list_provided = ["C","E","M","P","S","T"]

#### get all data -- design matrix, steady state abundances, and list of wells present

opt = bpdn.BPDN.Options({'Verbose': False, 'MaxMainIter': 1000,
                         'RelStopTol': 1e-10, 'AutoRho': {'RsdlTarget': 1.0}})

### hyperparameter values in LASSO 
n = 2**6
random_state = 33
cv_splitsl = [3,5]

In [6]:
sstv3 = pd.read_pickle("starch-sanchez-relative.pkl")

In [7]:
wellsp = list(sstv3.index)

In [8]:
dataTR = pd.read_pickle("traindataRELss.pkl")
dataTE = pd.read_pickle("testdataRELss.pkl")

In [9]:
list_present = wellsp
err = []
errlin = [] 
for cv_splits in cv_splitsl: 
    datatrain = dataTR[dataTR["k-fold"]==cv_splits]
    datatest = dataTE[dataTE["k-fold"]==cv_splits]
    for speciesc in range(6):
        datatousetrain = datatrain[datatrain["species"]==speciesc]
        datatousetest = datatest[datatest["species"]==speciesc]
        df2 = pd.DataFrame(species_ones[speciesc])
        idxlist = list(df2[0].values)
        abunvec = pd.DataFrame(sstv3[sp_list_provided[speciesc]][list(set(wellsp).intersection(species_ones[speciesc]))])
        pp = list(abunvec.index)
        redlist = []
        for kk in pp:
            redlist.append(idxlist.index(kk))
        abunvec.index = redlist
        for lmda in laml:
            for iterii in range(cv_splits):
                rinew = datatousetrain[datatousetrain["fold number"]==iterii]["train data"].values[0]
                rileft = datatousetest[datatousetest["fold number"]==iterii]["test data"].values[0]
                y3 = abunvec.T[rinew].T[sp_list_provided[speciesc]].values                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
                D = iHiVred[rinew,:]
                s2 = np.asarray([y3]).astype(float)
                s3 = s2.T
                b = bpdn.BPDN(D, s3, lmda, opt)
                x = b.solve()
                yrecon = np.dot(iHiVred,x.ravel())
                err.append([lmda,speciesc,yrecon[rileft],rileft,rinew,cv_splits])
                y3 = abunvec.T[rinew].T[sp_list_provided[speciesc]].values                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
                D = Xred[rinew,:]
                s2 = np.asarray([y3]).astype(float)
                s3 = s2.T
                b = bpdn.BPDN(D, s3, lmda, opt)
                x = b.solve()
                yrecon = np.dot(Xred,x.ravel())
                errlin.append([lmda,speciesc,yrecon[rileft],rileft,rinew,cv_splits])


In [10]:
errdf= pd.DataFrame(err)
errlindf = pd.DataFrame(errlin)


errdf.to_pickle("rel_CS_WH.pkl")
errlindf.to_pickle("rel_CS_sigma.pkl")